In [7]:
import numpy as np
import random
import matplotlib
matplotlib.use('Agg')  # noqa
import matplotlib.pyplot as plt

In [14]:
# Input Params

## Human Params ##
avg_age = 7300  # days - AKA 20S years
human_death_rate = 1/3650  # 1 human death every 10 yrs (for now, inaccurate)
inf_human_death_rate = 1/7  # An infected person dies in 7 days (for now, inaccurate)

## Mosquito Params ## 
mean_bites = 0.8  # average proportion of mosquitoes that successfully bite at each time step
stdev_bites = 0.1  # stdev of mean_bites
mosquito_death_rate = 0.1  # mosquito life span is ~10 days

## Time Params ##
t_max = 3650

## Infection Params ##
protection = 0.4  # proportion of reduced risk to infection after one infectiona
p_human_inf = 0.8  # avg prob that someone contractions malaria from an infected mosquito bite
p_mosquito_inf = 0.7  # prob that mosquito contracts malaria from an infected human

In [3]:
# Initializing Peoples Array based off Params
def init_peoples(Nh, Ih, avg_age):
    peoples = np.array([['person', 'infection', 'inf_age', 'prev_infs', 'age']])

    # Generate array
    for h in range(Nh):
        # Creating a random age w/ average of ave_age days
        rand_age = np.random.normal(avg_age, 1825)
        rand_age = np.round(rand_age).astype(int)
        new_person = [h+1, 0, 0, 0, rand_age]  # Assuming no one in the pop has been infected before?
        peoples = np.vstack([peoples, new_person])

    # If we choose to start the pop with initial infection
    for i in range(Ih):
        people_list = list(range(1, Nh+1))
        rand_person = random.choice(people_list)
        # An already infected person is chosen
        while int(peoples[rand_person][1]) == 1:  # NOTE: I want the first row to be string, but that means all entries are strings
            # Pick a new person
            rand_person = random.choice(people_list)

        # Person isn't already infected
        if int(peoples[rand_person][1]) == 0:
            peoples[rand_person][1] = 1
            # random generate an age of infection
            rand_age = np.random.normal(7, 4)
            rand_age = np.round(rand_age).astype(int)
            peoples[rand_person][2] = rand_age
    return peoples

In [4]:
def mosquito_bite_sim(mean_bites, stdev_bites, Nh, Nm):
    mosquito_bite_frac = np.random.normal(mean_bites, stdev_bites)  # rand generate proportion of mosquitoes that bite
    mosquito_bite_frac = max(0, min(1, mosquito_bite_frac))  #if mosquito_bite_frac is less than 0 --> 0, and if mosquito_bite_frac > 1 --> 1
    mosquito_bites = np.round(mosquito_bite_frac*Nm).astype(int)  # find total N of mosquitoes that bite
    bitten_list = [random.randint(1, Nh) for _ in range(mosquito_bites)]  # generate random list containing values 1 to Nh that represents the people that have been randomly bitten
    return bitten_list

In [15]:
# INITIALIZE POPULATION
Nh = 1000  # Total Human Population size
Ih = 0  # Num of Infected Humans to start

Nm = 100  # Total Mosquito Pop Size
Im = 20  # Num of Infected Mosquitoes to start

peoples = init_peoples(Nh, Ih, avg_age)

Ih_vs_t = []
Im_vs_t = []

for t in range(t_max):
    Ih_count = 0
    # print(f'******************************* DAY: {t} *******************************')
    healthy_bitten_list = mosquito_bite_sim(mean_bites, stdev_bites, Nh, Nm-Im)
    infected_bitten_list = mosquito_bite_sim(mean_bites, stdev_bites, Nh, Im)
    # print(f'People bitten by healthy mosquitoes: {healthy_bitten_list}')
    # print(f'People bitten by infected mosquitoes: {infected_bitten_list}')
    for h in peoples[1:]:  # iterate all rows, excluding the first row bc that is just headers
        # INFECTED HUMAN RECOVERY
        if int(h[2]) == 14:
            h[1] == '0'
            h[2] == '0'
            h[3] = str(int(h[3]) + 1)
    
        # UNINFECTED HUMAN IS BITTEN BY INFECTED MOSQUITO
        if int(h[0]) in infected_bitten_list and int(h[1]) == 0:
            # print('\n\nUNINFECTED HUMAN IS BITTEN BY INFECTED MOSQUITO')
            n_prev_inf = int(h[3])  # find num of prev infections
            protection_factor = 1-(1-protection)**n_prev_inf
            # print(f'Person {h[0]}s num of prev infections = {n_prev_inf}, Protection factor = {protection_factor}')
            rand_contraction = np.random.rand()  # gen rand num between 0 and 1
            if rand_contraction <= p_human_inf*(1-protection_factor):
                h[1] = '1'  # person becomes infected
                # print(f'PERSON {h[0]} HAS MALARIA NOW')
        
        # INFECTED HUMAN IS BITTEN BY HEALTHY MOSQUITO
        if int(h[0]) in healthy_bitten_list and int(h[1]) == 1:
            # print('\nA HEALTHY MOSQUITO BIT AN INFECTED HUMAN')
            rand_contraction = np.random.rand()  # gen rand num between 0 and 1
            if rand_contraction <= p_mosquito_inf:
                Im = Im + 1
                # print(f'MOSQUITO HAS MALRIA NOW, Infected Mosquito Count = {Im}')

        # HUMAN AGING UP
        new_age = int(h[4]) + 1
        h[4] = str(new_age)
        
        # HUMAN DEATH
        if int(h[1]) == 1:
            rand_death = np.random.rand()
            # infected person dies and replaced with a fresh susceptible
            if rand_death <= inf_human_death_rate:
                h[1] = '0'
                h[2] = '0'
                h[3] = '0'
                h[4] = '0'
            else:  # infected person lives to see another day
                h[2] = str(int(h[2])+1)
        if int(h[1]) == 0:
            rand_death = np.random.rand()
            if rand_death <= human_death_rate:
                h[1] = '0'
                h[2] = '0'
                h[3] = '0'
                h[4] = '0'
        if int(h[1]) == 1:
            Ih_count = Ih_count+1
    # MOSQUITO DEATH
    Im = (1-mosquito_death_rate)*Im
    Ih_vs_t.append(Ih_count)

In [16]:
t_list = []
for t in range(t_max):
    t_list.append(t)

fig, ax = plt.subplots()
ax.plot(t_list,Ih_vs_t)
plt.savefig('test.png')